In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [3]:
fake = pd.read_csv('Datasets\Fake.csv')
true = pd.read_csv('Datasets\True.csv')

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Irteja\AppData\Local\Temp\ipykernel_15920\3296890919.py:1: SyntaxWarning: invalid escape sequence '\F'
  fake = pd.read_csv('Datasets\Fake.csv')
C:\Users\Irteja\AppData\Local\Temp\ipykernel_15920\3296890919.py:2: SyntaxWarning: invalid escape sequence '\T'
  true = pd.read_csv('Datasets\True.csv')


In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class'] = 0
true['class'] = 1

In [7]:
df = pd.concat([fake, true]).sample(frac=1, random_state=42)

In [8]:
# Combine title + text into one column
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('') + " " + df['subject'].fillna('')

In [9]:
df.sample(10)

,title,text,subject,date,class,content
7647,Clinton says confident new emails will not cha...,"DES MOINES, Iowa (Reuters) - U.S. Democratic p...",politicsNews,"October 28, 2016",1,Clinton says confident new emails will not cha...
20098,Ukraine gives cautious welcome to Putin's peac...,BERLIN (Reuters) - Ukraine on Thursday welcome...,worldnews,"September 14, 2017",1,Ukraine gives cautious welcome to Putin's peac...
15597,China releases draft law to expand power of ne...,BEIJING (Reuters) - China s legislature on Mon...,worldnews,"November 6, 2017",1,China releases draft law to expand power of ne...
15511,Jailed Hong Kong democracy activists win last ...,HONG KONG (Reuters) - A Hong Kong court on Tue...,worldnews,"November 7, 2017",1,Jailed Hong Kong democracy activists win last ...
11299,North Korean defectors may have been exposed t...,SEOUL (Reuters) - At least four defectors from...,worldnews,"December 27, 2017",1,North Korean defectors may have been exposed t...
19826,Spain's prosecutor warns over Catalonia refere...,BARCELONA (Reuters) - Spanish authorities on S...,worldnews,"September 17, 2017",1,Spain's prosecutor warns over Catalonia refere...
16235,"China's Guangdong gets new party boss, former ...",BEIJING (Reuters) - China on Saturday appointe...,worldnews,"October 28, 2017",1,"China's Guangdong gets new party boss, former ..."
12181,Trump aide Greenblatt heads to Israel after Je...,WASHINGTON (Reuters) - President Donald Trump ...,worldnews,"December 15, 2017",1,Trump aide Greenblatt heads to Israel after Je...
3810,Kushner Cos says to skip China marketing push ...,SHANGHAI (Reuters) - The company owned by the ...,politicsNews,"May 12, 2017",1,Kushner Cos says to skip China marketing push ...
7320,"Scared by Trump, some migrants on Mexico borde...","NOGALES, Mexico (Reuters) - Donald Trump’s ele...",politicsNews,"November 11, 2016",1,"Scared by Trump, some migrants on Mexico borde..."


In [10]:
df = df.drop(["title","subject","date","text"], axis = 1)

In [11]:
df.head()

,class,content
22216,0,Ben Stein Calls Out 9th Circuit Court: Committ...
4436,1,Trump drops Steve Bannon from National Securit...
1526,1,Puerto Rico expects U.S. to lift Jones Act shi...
1377,0,OOPS: Trump Just Accidentally Confirmed He Le...
8995,1,Donald Trump heads for Scotland to reopen a go...


In [12]:
df.reset_index(inplace=True)

In [13]:
# --- Text Cleaning ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [14]:
df['cleaned'] = df['content'].apply(clean_text)

In [15]:
df.sample(10)

,index,class,content,cleaned
40753,7352,0,This Christian Mom Thought It Was A Good Idea...,this christian mom thought it was a good idea ...
17181,4740,1,No. 2 House Republican says healthcare bill de...,no house republican says healthcare bill debat...
17068,14406,0,WHY AMERICANS SHOULD Care That Facebook’s CEO ...,why americans should care that facebook’s ceo ...
10168,13168,0,HYSTERICAL! HILLARY’S SPEECH HIJACKED By Conse...,hysterical hillary’s speech hijacked by conser...
17500,19346,1,Slovenian PM cancels Croatia visit over mariti...,slovenian pm cancels croatia visit over mariti...
4607,17658,1,Britain's Boris Johnson tells EU: put a tiger ...,britains boris johnson tells eu put a tiger in...
7400,15718,0,(VIDEO) CLINTON FOUNDATION HOSTS FUNDRAISERS ‘...,video clinton foundation hosts fundraisers ‘wi...
1888,17925,0,REVEALED: HOW BARACK OBAMA IS FUNDING The “Res...,revealed how barack obama is funding the “resi...
37345,8123,0,Watch: Trump Doesn’t Know What To Do During M...,watch trump doesn’t know what to do during mom...
42159,781,0,Trump’s America: White Supremacist Blogger Gi...,trump’s america white supremacist blogger give...


In [16]:
x =df["cleaned"]
y=df["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [17]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [18]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [19]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9876169265033408

In [20]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5863
           1       0.98      0.99      0.99      5362

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [21]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']